In [1]:
#!/usr/bin/env python

from fanuc_pcdk_client import FanucPCDKClient
from robot_loc_world import RobotLocWorld
from robot_loc_joint import RobotLocJoint
from time import sleep
from copy import deepcopy
import sys

ROBOT_SPEED = 30
ROBOT_SPEED_LINEAR = 45

In [61]:
# Robot init
robot = FanucPCDKClient()
robot.connect()

# Run teach pendant program.
robot.run_program('PCDK')
robot.reset_alarms()

# Adjust the speed of the robot
robot.set_speed(ROBOT_SPEED)

Connecting to robot at 192.168.1.1
Robot Status: Connected



In [3]:
valid_letters = ['s', 'l', 'o', 'w', 'u', 'b']

In [31]:
# MOVE ROBOT TO STARTING WRITING POSE (PEN TO PAPER POSE) AND RUN THIS CELL. 
# THEN USE THE INFORMATION TO FILL OUR THE 'write_start_pose' and 'write_start_pose_joints' 
# VARIABLES IN THE NEXT CELL 

# Retrieve the current world XYZWPR coordinates of the robot
loc_world = robot.get_position("world")
print(f'Current world location: {loc_world}\n')

# Retrieve the current joint coordinates of the robot
loc_joint = robot.get_position("joint")
print(f'Current joint location: {loc_joint}\n')

Current world location: x=268.136; y=-347.037; z=-222.553; w=179.976; p=2.054; r=0.943; f=False; u=True; t=True

Current joint location: j1=-51.411; j2=49.091; j3=-35.697; j4=1.990; j5=-55.556; j6=51.211



====== #1
Current world location: x=417.779, y=-97.121, z=-105.294, w=179.009, p=-16.645, r=-2.775, f=False, u=True, t=True

Current joint location: j1=-14.393, j2=21.375, j3=-27.787, j4=-3.260, j5=-45.744, j6=13.410

====== #2
Current world location: x=272.799, y=-348.465, z=-102.341, w=-179.668, p=1.804, r=-38.450, f=False, u=True, t=True

Current joint location: j1=-51.892, j2=39.761, j3=-16.327, j4=0.100, j5=-75.505, j6=13.410


====== #3
Current world location: x=268.136, y=-347.037, z=-222.553, w=179.976, p=2.054, r=0.943, f=False, u=True, t=True

Current joint location: j1=-51.411, j2=49.091, j3=-35.697, j4=1.990, j5=-55.556, j6=51.211

====== #4


In [35]:
p1_w = RobotLocWorld(x=417.779, y=-97.121, z=-105.294, w=179.009, p=-16.645, r=-2.775, f=False, u=True, t=True)
p1_j = RobotLocJoint(j1=-14.393, j2=21.375, j3=-27.787, j4=-3.260, j5=-45.744, j6=13.410
)

p2_w = RobotLocWorld(x=272.799, y=-348.465, z=-102.341, w=-179.668, p=1.804, r=-38.450, f=False, u=True, t=True)
p2_j = RobotLocJoint(j1=-51.892, j2=39.761, j3=-16.327, j4=0.100, j5=-75.505, j6=13.410)

p3_w = RobotLocWorld(x=268.136, y=-347.037, z=-222.553, w=179.976, p=2.054, r=0.943, f=False, u=True, t=True)
p3_j = RobotLocJoint(j1=-51.411, j2=49.091, j3=-35.697, j4=1.990, j5=-55.556, j6=51.211)

In [56]:
# Trajectories/offsets for each letter

# write start location
# SET POSE ACCORDINLY
write_start_pose = RobotLocWorld(x=417.797, y=-97.137, z=-248.736, w=-178.712, p=-5.532, r=-3.217, f=False, u=True, t=True)
# SET JOINT ANGLES ACCORDINGLY
write_start_pose_joints = RobotLocJoint(j1=-14.395, j2=46.102, j3=-44.810, j4=-3.629, j5=-40.067, j6=13.913)

# Pen location # maybe save the joint angles for this one
pen_loc = None
# Pick up pen traj
pen_traj = [p1_w, p2_w, p3_w, 'close', p2_w, p1_w]

# Return pen traj - Not actually needed

down_z = write_start_pose.z
up_z = write_start_pose.z + 10 # rises 1cm


letter_offset = [(0, 50)]

# tuples in (+/- x, +/- y)
# S traj
s_offset = ['down', (0, 30), (0, -30), (30, 0), (0, 30), (30, 0), (0, -30), 'up', (-60, 0)]

# L traj
l_offset = ['down', (60, 0), (0, 30), 'up', (0, -30), (-60, 0)]

# O traj
o_offset = ['down', (60, 0), (0, 30), (-60, 0), (0, -30), 'up']

# U traj
u_offset = ['down', (60, 0), (0, 30), (-60, 0), 'up', (0, -30)]

# W traj
w_offset = ['down', (60, 0), (0, 15), (-30, 0), (30, 0), (0, 15), (-60, 0), 'up', (0, -30)]

# W traj alt
w_offset = ['down', (60, 7.5), (-30, 7.5), (30, 7.5), (-60, 7.5), 'up', (0, -30)]

# B traj
b_offset = ['down', (60, 0), (0, 30), (-30, 0), (0, -15), (0, 15), (-30, 0), (0, -30), 'up']

offsets = {'s': s_offset, 'l': l_offset, 'o': o_offset, 'w': w_offset, 'u': u_offset, 'b': b_offset}

# trajectories
trajectories = {'s': [], 'l': [], 'o': [], 'w': [], 'u': [], 'b': []}

In [23]:
def home_routine():
    home_pose = deepcopy(write_start_pose)
    curr_pose = robot.get_position("world")
    # move up
    curr_pose.z = -209.206
    robot.move(curr_pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)
    home_pose.z = curr_pose.z
    robot.move(home_pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)

> Already at location; skipping move
> Already at location; skipping move


In [16]:
def compute_trajectory(pose, letter):
    traj = []
    if(letter == 'offset'):
        target_offsets = letter_offset
    else:
        target_offsets = offsets[letter.lower()]
    curr_pose = deepcopy(pose)
    for offset in target_offsets:
        if(offset == 'down'):
            curr_pose.z = down_z
        elif(offset == 'up'):
            curr_pose.z = up_z
        elif(type(offset) == tuple):
            curr_pose.x += offset[0]
            curr_pose.y += offset[1]
        goal = deepcopy(curr_pose)
        traj.append(goal)
    return traj

In [7]:
def pickup_pen():
    # Use to complete pen pick trajectory
    for waypoint in pen_traj:
        if(type(waypoint) == RobotLocJoint):
            robot.move_joint(waypoint)
        elif(type(waypoint) == RobotLocWorld):
            robot.move(waypoint, move_linear=True, speed=ROBOT_SPEED_LINEAR)
        elif(waypoint == 'pick'):
            cur_pose = robot.get_position("world")
            curr_pose.z -= ABOVE_PEN_HEIGHT
            robot.move(curr_pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)
            robot.close_gripper()
            curr_pose.z += 2*ABOVE_PEN_HEIGHT # CHANGE ACCORDINLY
            robot.move(curr_pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)

In [50]:
def get_pen():
    # Use to complete pen pick trajectory
    robot.close_gripper()
    for waypoint in pen_traj:
        if(type(waypoint) == RobotLocJoint):
            robot.move_joint(waypoint)
        elif(type(waypoint) == RobotLocWorld):
            robot.move(waypoint, move_linear=True, speed=ROBOT_SPEED_LINEAR)
        elif(waypoint == 'close'):
            robot.open_gripper()


In [45]:
home_routine()

In [52]:
robot.close_gripper()

In [53]:
def execute_writing(word):
    print("\n\n===============    STARTING WRITING    ===============")
    home_routine()
    get_pen()
    for letter in word:
        curr_pose = robot.get_position("world")
        trajectory = compute_trajectory(robot.get_position("world"), letter)
        
        for waypoint in trajectory:
            robot.move(waypoint, move_linear=True, speed=ROBOT_SPEED_LINEAR)
        off_traj = compute_trajectory(robot.get_position("world"), 'offset')
        for waypoint in off_traj:
            robot.move(waypoint, move_linear=True, speed=ROBOT_SPEED_LINEAR)
    home_routine()
    robot.close_gripper()

In [62]:
terminate = False
user_input = 0
started = False
while not terminate:
    try:
        if(not started):
            print("===============  WELCOME TO THE ROBOTIC (LIMTIED) TYPEWRITER  ===============\n")
            started = True
            
        user_input = (input("What 3-letter word would you like to write? (press q to quit)\
                        \nValid letters include S L O W U B\
                        \n> "))
        
        if(not (user_input == 'q')):
            if (len(user_input) > 3):
                raise ValueError
                
            for letter in user_input:
                if(letter.lower() not in valid_letters):
                    raise ValueError
                
        if(user_input == 'q'):
            print("\n===============    TERMINATING WRITING    ===============")
            terminate = True
            break
        else:
            '''
            ACTUAL WRITING STUFF GOES HERE
            '''
            execute_writing(user_input)
            
    except ValueError:
        print("Invalid word, please try again")

===============  WELCOME TO THE ROBOTIC (LIMTIED) TYPEWRITER  ===============

What 3-letter word would you like to write? (press q to quit)                        
Valid letters include S L O W U B                        
> LOW


===============    STARTING WRITING    ===============
What 3-letter word would you like to write? (press q to quit)                        
Valid letters include S L O W U B                        
> q

===============    TERMINATING WRITING    ===============
